In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised = True)
import numpy as np
train_data , test_data = imdb['train'],imdb['test']
training_sentences=[]
training_labels = []
testing_sentences=[]
testing_labels = []

In [3]:
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen= max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen= max_length, truncating = trunc_type)


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss = "mean_squared_error",optimizer = "sgd", metrics = ['accuracy'])
model.fit(padded,training_labels_final,epochs = 5,validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 10s 402us/sample - loss: 0.2405 - accuracy: 0.6158 - val_loss: 0.2416 - val_accuracy: 0.5977
Epoch 2/5
25000/25000 [==============================] - 7s 283us/sample - loss: 0.2383 - accuracy: 0.6229 - val_loss: 0.2394 - val_accuracy: 0.6088
Epoch 3/5
25000/25000 [==============================] - 7s 287us/sample - loss: 0.2356 - accuracy: 0.6302 - val_loss: 0.2369 - val_accuracy: 0.6163
Epoch 4/5
25000/25000 [==============================] - 7s 278us/sample - loss: 0.2326 - accuracy: 0.6386 - val_loss: 0.2341 - val_accuracy: 0.6228
Epoch 5/5
25000/25000 [==============================] - 7s 261us/sample - loss: 0.2291 - accuracy: 0.6478 - val_loss: 0.2310 - val_accuracy: 0.6364


In [24]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[2]))
print(padded[2])
print(training_sentences[2])

<OOV> photographs the <OOV> rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of dawson city <OOV> to <OOV> themselves a <OOV> yes a <OOV> and to <OOV> the law themselves then <OOV> battling it out on the streets for control of the town br br nothing even remotely resembling that happened on the canadian side of the border during the <OOV> gold rush mr mann and company appear to have mistaken dawson city for <OOV> the canadian north for the american wild west br br canadian viewers be prepared for a <OOV> madness type of enjoyable
[   1 6175    2    1 4916 4029    9    4  912 1622    3 1969 1307    3
 2384 8836  201  746  361   15   34  208  308    6   83    8    8   19
  214   22  352    4    1  990    2   82    5 3608  545    1    6    1
  539    4    1  434    4    1    3    6    1    2 1176  539   95    1
 8111   10   46   22    2 1996   16 1153    5  

In [28]:
print(training_labels[0])

0


In [33]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 390       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_________________________________________________________________
